In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler

## 1. Import dataset

In [ ]:
index_col_names=['unit_id','time_cycle']
operat_set_col_names=['oper_set{}'.format(i) for i in range(1,4)]
sensor_measure_col_names=['sm_{}'.format(i) for i in range(1,22)]
all_col=index_col_names+operat_set_col_names+sensor_measure_col_names
print(all_col)

In [ ]:
train_df=pd.read_csv('../input/nasa-cmaps/CMaps/train_FD001.txt',delim_whitespace=True,names=all_col)
train_df

In [ ]:
train_df[train_df.unit_id==1]

In [ ]:
train_df.info()

The dimensions of test set and RUL data are different, because RUL file assigns one value per unit. Then, we assign the RUL value to the last data point for each unit and adding one to each row above 

In [ ]:
test_df=pd.read_csv('../input/nasa-cmaps/CMaps/test_FD001.txt',delim_whitespace=True,names=all_col)
test_df.head()

In [ ]:
test_df.info()

In [ ]:
y_true=pd.read_csv('../input/nasa-cmaps/CMaps/RUL_FD001.txt',delim_whitespace=True,names=['RUL'])
y_true['unit_id']=y_true.index+1
y_true.head()

In [ ]:
y_true

In [ ]:
test_df[test_df.unit_id==1]

## 2. Obtain RUL

## 2.1 training set

In [ ]:
#find maximum time cycle for each unit d
max_time_cycle=train_df.groupby('unit_id')['time_cycle'].max()
print(max_time_cycle)
rul = pd.DataFrame(max_time_cycle).reset_index()
rul.columns = ['unit_id', 'max']
rul.head()

In [ ]:
rul

In [ ]:
#merge train_df with rul dataframe based on number unit, that is the column join, using the left join
train_df = train_df.merge(rul, on=['unit_id'], how='left')

In [ ]:
train_df['RUL'] = train_df['max'] - train_df['time_cycle']
train_df.drop('max', axis=1, inplace=True)

In [ ]:
train_df[train_df.unit_id==1].iloc[:,[1,-1]]

## 2.2 test set

In [ ]:
test_df['RUL']=0
for i in range(1,101):
    test_df.loc[test_df.unit_id==i,'RUL']=range(int(y_true.RUL[y_true.unit_id==i])+len(test_df[test_df.unit_id==i])-1,
                                      int(y_true.RUL[y_true.unit_id==i])-1,-1)

In [ ]:
test_df.iloc[:,[0,1,-1]]

In [ ]:
y_true

In [ ]:
#check if it's correct changing unit_id
y_true
test_df.loc[test_df.unit_id==5,['unit_id','time_cycle','RUL']]

## 3. Feature Selection

In [ ]:
train_df.head()

In [ ]:
train_df[index_col_names].corr()

In [ ]:
train_df[operat_set_col_names].corr()

In [ ]:
train_df[sensor_measure_col_names].corr()

From correlation matrix, we notice:
* oper_set3 is not correlated with the other variables
* sensor 1, 5,10,16,18,19

In [ ]:
train_df.iloc[:,1:-1].corr()

In [ ]:
for v_idx,v in enumerate(all_col[1:]):
    for i in train_df['unit_id'].unique():
        plt.plot('RUL',v,data=train_df[train_df['unit_id']==i])
    plt.xlabel('RUL')
    plt.ylabel(v)
    plt.show()    

In [ ]:
fig,ax=plt.subplots(13,2,figsize=(12,20))
fig.tight_layout()
r,c=0,0
for v_idx,v in enumerate(all_col[1:]):
    for i in train_df['unit_id'].unique():
        ax[r][c].plot('RUL',v,data=train_df[train_df['unit_id']==i])
            
    ax[r][c].set_xlabel('RUL')
    ax[r][c].set_ylabel(v) 
    if c<1:
        c+=1
    elif c==1:
         r+=1
         c-=1

In [ ]:
cols_drop=['oper_set3','sm_1','sm_5','sm_6','sm_10','sm_14','sm_16','sm_18','sm_19']
train_df = train_df.drop(cols_drop, axis = 1)
test_df = test_df.drop(cols_drop, axis = 1)

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
train_df.corr()

## 4. Data Preprocessing

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
features=list(train_df.columns[1:-1])

In [ ]:
features

In [ ]:
min_max_scaler = MinMaxScaler(feature_range=(-1,1))

train_df[features] = min_max_scaler.fit_transform(train_df[features])
test_df[features] = min_max_scaler.fit_transform(test_df[features])

In [ ]:
test_df.head()

In [ ]:
test_df.describe()

In [ ]:
train_df

In [ ]:
train_df.drop(['unit_id','RUL'],axis=1).columns

In [ ]:
X_train = train_df.drop(['unit_id','RUL'],axis=1).values
y_train = train_df['RUL'].values

In [ ]:
X_test = test_df.drop(['unit_id','RUL'],axis=1).values
y_test = test_df['RUL'].values

## 5. Gradient Boosting

In [ ]:
len(X_train[0])

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

In [ ]:
reg = GradientBoostingRegressor(max_features='sqrt',n_estimators=800,random_state=42)
reg.fit(X_train, y_train)

# predict and evaluate
y_hat_train = reg.predict(X_train)
print('training RMSE: ',np.sqrt(mean_squared_error(y_train, y_hat_train)))

print()

y_hat_test = reg.predict(X_test)
print('test RMSE: ',np.sqrt(mean_squared_error(y_test, y_hat_test)))

In [ ]:
reg.feature_importances_

In [ ]:
importances = reg.feature_importances_
sorted_index=np.argsort(importances)[::-1]
x=range(len(importances))
labels=np.array(train_df.drop(['unit_id','RUL'],axis=1).columns)[sorted_index]

plt.bar(x,importances[sorted_index],tick_label=labels)

plt.xticks(rotation=90)
plt.show()

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lm = LinearRegression()
lm.fit(X_train, y_train)

# predict and evaluate
y_hat_train = lm.predict(X_train)
print(np.sqrt(mean_squared_error(y_train, y_hat_train)))

y_hat_test = lm.predict(X_test)
print(np.sqrt(mean_squared_error(y_test, y_hat_test)))

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
m = RandomForestRegressor(max_features='sqrt',n_estimators=600,random_state=42)
m.fit(X_train, y_train)

y_hat_train = m.predict(X_train)
print(np.sqrt(mean_squared_error(y_train, y_hat_train)))

y_hat_test = m.predict(X_test)
print(np.sqrt(mean_squared_error(y_test, y_hat_test)))

## Hyperparameter optimization

In [ ]:
from sklearn.model_selection import RandomizedSearchCV,StratifiedKFold,KFold
from scipy.stats import randint

In [ ]:
reg = GradientBoostingRegressor(random_state=0)
cv = KFold(n_splits=5, random_state=0, shuffle=True)

param_space={
        'max_depth': randint( 3, 100),
        'n_estimators': randint(550, 1000)
    }

reg_cv = RandomizedSearchCV(reg, param_space, cv=cv)
# Fit it to the data
reg_cv.fit(X_train, y_train)
# Print the tuned parameters and score
print("Tuned {} Parameters: {}".format('gradient boosting',reg_cv.best_params_))
print("Best score is {}".format(reg_cv.best_score_))